# Transient

#### Dimensions:
50 by 50 by 1
dx = dy = 10 m
dz = 50 m
the model is unconfined
#### Subsurface Properties:
Homogeneous
K = 0.1 m/day in x, y and z
Porosity = 0.35
Specific yield = 0.3
Storage coefficient=0.001
#### Boundary Conditions:
Right boundary is a constant head of 30 m relative to the datum, which is located at the bottom of the domain.
Left boundary is a constant head of 50 m relative to the datum, which is located at the bottom of the domain.
Recharge occurs at a rate of 1E-5 m/d uniformly across the domain
There is no ET in the domain


#### Initial Conditions:
Set the initial head to 50 everywhere, which will be replaced by the initial steady state solution
#### Pumpinmg
Well located at row, column 20,20
There is no pumping during the initial steady state period
Case 1: pumps at a rate of 500 m3/d for 90 days, then zero for remaining 270 days of the year for 100 years. (Assume a 360 day year made of 12 30-day months for simplicity of plotting.)
### The Challenges
The gradient is not uniform for the initial steady state conditions - discuss the influences of recharge and the unconfined condition on this nonlinearity

Determine if the system has reached steady state - consider a point at the well and another at the center of the domain.
Find the zone of influence of the well defined in two ways:
Based on the drawdown from the initial steady state to the end of simulation time (end of final no-pumping stress period).
Based on the drawdown from the end of the last pump-on stress period to the end of simulation time.

How long does it take a point at the center of the domain to reach steady state. At that point, explain how you could divide the domain into a steady and transient part and solve each separately.

Find a constant pumping rate (same throughout the year) that matches the head time series at the middle of the domain.

Find a constant pumping rate (same throughout the year) that matches the head time series at the well, leaving only a regular, repeating seasonal residual. Are the two pumping rates the same?

Discuss the sources of water captured by this well. If you're up for a challenge, calculate them for the final pump-on period!

Discuss how you would define the capture zone of the well. How is it different than our definitions of capture zone so far in the course?

### 1. Setup the environment

In [ ]:
#import the basics
import flopy
import numpy as np
import matplotlib as mp
import os
import pandas as pd

#additional analysis tools
import flopy.utils.binaryfile as bf
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import csv

#jupyter specific--included to show plots in notebook
%matplotlib inline

### 2. Create the model object

In [ ]:
modelname = "Transient"
moddir=r"C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\Course_Materials\Assignments\mf2005"
m = flopy.modflow.Modflow(modelname, exe_name = moddir)

In [ ]:
#optional delete old run files
run = 'n'
if run == 'y':
    # file_extensions = ['.dis','.bas','.lpf','.oc','.pcg','.wel']
    file_extensions = ['.dis','.bas','.bcf','.oc','.pcg', '.wel', 'rch','evt']
    for ext in file_extensions:
        os.remove(modelname + ext)

### 3. Setup the discretization (DIS package)
For more details on DIS package inputs refer to : https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mfdis.html

##### 3.1 Spatial discretization

In [ ]:

nrow = 50 #number of rows
ncol = 50 #number of columns
nlay = 1  #number of layers

dx= 10 #width of grid cells in x direction 
dy= 10 #width of grid cells in y direction 

Lx = ncol*dx  #width of domain in x
Ly = nrow*dy #width of domain in y
ztop = 50.    #top elevation 
zbot = 0.     #bottom elevation

dz = (ztop - zbot) / nlay #cell height in z direction

domain_centerpt = (0, int(nrow/2), int(ncol/2))   # used for plotting

##### 3.2 Temporal discretization

In [ ]:
nper = 1 #number of stress periods 
steady = [True] #list noting the stress period type


sp_peryr = 2        # periods for seasonality (two stress periods per year)
sp1_len = 90       # Length of stress period 1 in days
sp2_len = 360-sp1_len      # Length of stress period 2 in days
sim_years = 100       #number of years for transient simulation 
nper = int(sp_peryr*sim_years + 1) # total number of stress periods the +1 is for the steady state start

#setup the steady state array
steady = np.ones(nper, dtype=bool)*False     # steady-state or transient flag for each stress period (boolean array)
steady[0] = True    # initial stress period is steady-state

#make an array of period lengths
perlen=np.tile([sp1_len, sp2_len],sim_years)
perlen=np.append(10,perlen) #add a period of length 10 at the start for the steady state simulation

nstp=perlen/30 # set monthly time steps per stress period to get monthly results
nstp[0]=1 #for the steady state solution we just want one output
nstp = nstp.astype(int)

elyears = (np.cumsum(perlen)-perlen[0])/360

##### 3.3 Create DIS object

In [ ]:
#create flopy discretization object, length and time are meters (2) and days (4)
dis = flopy.modflow.ModflowDis(model=m, nlay=nlay, nrow=nrow, ncol=ncol, 
                               delr=dx, delc=dy, top=ztop, botm=zbot, 
                               itmuni = 4, lenuni = 2, 
                               nper=nper, steady=steady)

##### 3.4 Plot the grid
For help on this function see: https://flopy.readthedocs.io/en/3.3.2/source/flopy.plot.plotutil.html

In [ ]:
#use flopy to plot the grid of model 'm'
modelmap = flopy.plot.PlotMapView(model=m, layer=0)
grid = modelmap.plot_grid() 
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.title('Grid', fontsize = 15, fontweight = 'bold')
plt.show()


### 4. Setup constant head boundaries, active cells and initial heads (BAS Package)
For more details on the BAS package see: https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mfbas.html

##### 4.1 Define cell activity: IBOUND

In [ ]:
#create ibound as array of ints = 1
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32) #integer array of dim (z,y,x), makes all cells active     

#set constant head boundary on the left
ibound[:,:,0] = -1          #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head 
ibound[:,:,-1] = -1          #replace leftmost column (all zs, all ys, first x) with -1 to indicate constant head

##### 4.2 Define starting head

In [ ]:
#setup initial heads as 70 everywhere, this value will be preserved on constant head boundaries  
H_init = np.ones((nlay, nrow, ncol), dtype=np.float32)  #float array (i.e. decimal), sets all heads to 1.0
H_init[:, :, :] = 50    # set all values to uniform initial condition
H_init[:, :, -1] = 30    #replace last (left) col with desired head value

##### 4.3 Create BAS object

In [ ]:
#create flopy bas object
bas = flopy.modflow.ModflowBas(m, ibound=ibound, strt=H_init)

In [ ]:
#plot grid and ibound
modelmap = flopy.plot.PlotMapView(model=m, layer=0)
grid = modelmap.plot_grid()
ib = modelmap.plot_ibound()
#add labels and legend
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.title('Ibound', fontsize = 15, fontweight = 'bold')
plt.legend(handles=[mp.patches.Patch(color='blue',label='Const. Head',ec='black'),
                   mp.patches.Patch(color='white',label='Active Cell',ec='black'),
                   mp.patches.Patch(color='black',label='Inactive Cell',ec='black')],
                   bbox_to_anchor=(1.5,1.0))
plt.show(modelmap)


# I'm not sure why this stopped working!  Can someone figure out why the right boundary doesn't show up as constant head??

### 5. Setup Layer Properties (LPF Package)¶
For more details on LPF package see: https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mflpf.html

##### 5.1 Set layer properties

In [ ]:
K_horiz = .1    #horizontal hydraulic conductivity 
K_vert = .1     #assign vertical hydraulic conductivity (along z axis)
n = 0.35        #assign porosity
Ss = 0.001      #assign storage coefficient
Sy = 0.03       #assign specific yield
uncnf = 0       #0=confined, >0 = unconfined

##### 5.2 Make LPF object

In [ ]:

# kept in case you want to use it later

#assigns horizontal and vertical Ks and saves cell-by-cell budget data
# lpf = flopy.modflow.ModflowLpf(m, laytyp=uncnf, hk=K_horiz, 
#                                vka=K_vert, ss=Ss,
#                                sy=Sy,storagecoefficient=True, ipakcb=53)

### 5b. Setup Cell-byCell Properties (BCF Package)¶
For more details on LPF package see: https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mfbcf.html

##### 5b.1 Set cell properties and show K in top layer

In [ ]:
K_vals = np.zeros((ncol,nrow,nlay))
K_vals[:,:,:] = 1             # homoogeneous background hydraulic conductivity 
#K_vals[10:15,10:15,:] = 0.1   # low K inclusion

#plot K values in top layer
fig1 = plt.figure(figsize=(10,10)) #create 10 x 10 figure
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.title('K in top layer', fontsize = 15, fontweight = 'bold')
plt.contourf(K_vals[:,:,0])
plt.colorbar()
plt.show()

##### 5b.2 Make BCF object

In [ ]:
#assigns horizontal and vertical Ks and saves cell-by-cell budget data
bcf = flopy.modflow.ModflowBcf(m, ipakcb=53, intercellt=0, laycon=3, trpy=1.0, hdry=-1e+30, iwdflg=0, wetfct=0.1, iwetit=1, ihdwet=0, tran=1.0, hy=K_vals[:,:,0], vcont=1.0, sf1=Ss, sf2=Sy, wetdry=-0.01, extension='bcf', unitnumber=None, filenames=None)

### 6. Setup outputs by stress period (OC Package)
For more details on OC Package: https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mfoc.html

##### 6.1 Setup outputs

In [ ]:
#create oc stress period data - save for each time step in each stress period. 
oc_spd = {}
for kper in range(nper):  
    for kstp in range(nstp[kper]):
        #print(kstp)
        oc_spd[(kper, kstp)] = ['save head','save drawdown','save budget','print head','print budget']

##### 6.2 Make OC object

In [ ]:
oc = flopy.modflow.ModflowOc(model=m, stress_period_data=oc_spd, compact=True)

### 7. Set the numerical solver (PCG Package)¶
We are using the Preconditioned Conjugate Gradient package:

Additionally, MODFLOW online describes these solvers: https://water.usgs.gov/ogw/modflow/MODFLOW-2005-Guide/index.html?oc.htm

And Flopy documents its communication with the PCG MODFLOW package: http://modflowpy.github.io/flopydoc/mfpcg.html

##### 7.1 Make the PCG object

In [ ]:
pcg = flopy.modflow.ModflowPcg(model=m)

### 8. Add wells for the constant flux boundary (WEL Package)
For more details on the WEL package see: http://modflowpy.github.io/flopydoc/mfwel.html

##### 8.1 Setup the well parameters (use this to set a defined flux boundary)

In [ ]:
#Setup alternating pumping values
well_loc = [0, 20, 20]
Q1 = 0
Q2 = -500
Q_val=np.tile([Q1, Q2],sim_years)
Q_val=np.append(Q1,Q_val) # add a period of length 10 at the start for the steady state simulation
                      
Q = {}
for p in range(0,nper):    #loop over stress periods
    Q[p]=Q_val[p]

well_spd = {}                                                                     # create empty dictionary for well info
for per in range(nper):                                                          # transient models, iterate over stress periods to assign well stresses
    well_spd[per] = [[well_loc[0], well_loc[1], well_loc[2], Q_val[per]]]     # assign well coord & pump rate for each stress period

##### 8.2 Make the WEL object

In [ ]:
wel = flopy.modflow.mfwel.ModflowWel(m, stress_period_data=well_spd)  #create object for WEL package

### 9. Add recharge (RCH Package)
For more details on the WEL package see: http://modflowpy.github.io/flopydoc/mfrch.html

##### 9.1 Setup recharge value

In [ ]:
#Recharge 
rech_zone = np.zeros((nrow,ncol))               #define an array of zeros of dim (nrow,ncol) - backgroun ET is zero everywhere
recharge = 5e-5 #m/day                          
#rech_zone[10:20,10:20] = recharge

rch = flopy.modflow.mfrch.ModflowRch(model=m, rech=rech_zone, ipakcb=53)     #create rch object

#####  9.2 Add evapotranspiration

In [ ]:
'''# ET - this is set up to show how you would add time-varying recharge in a localized area
ET_val1 = 0 #m/day
ET_val2 = 0.01 #m/day
ET_locxy=[[400, 900], [1500, 1600]] #lower left and upper right corners of ET in xy coordinate
extinction_depth = 50

ET_rows=(np.floor(nrow-ET_locxy[1][1]/dy),np.floor(nrow-ET_locxy[0][1]/dy)) #convert the y locations to rows
ET_cols=(np.floor(ET_locxy[0][0]/dx),np.floor(ET_locxy[1][0]/dx)) #convert the y locations to rows
ET_centerpt=(0, int(np.mean(ET_rows)),int(np.mean(ET_cols)))
ET_zone = np.zeros((nlay,nrow,ncol))     # define an array of zeroes the size of the model grid
ET_zone[0,int(ET_rows[0]):int(ET_rows[1]), int(ET_cols[0]):int(ET_cols[1])] = ET_val2  # assign ET rate to the ET zone

#Setup alternating ET values
ET_val=np.tile([ET_val1, ET_val2],sim_years)
ET_val=np.append(ET_val1,ET_val) # add a period of length 10 at the start for the steady state simulation
                      
ET = {}
for p in range(0,nper):    #loop over stress periods
    ET[p]=ET_zone*ET_val[p]

evt = flopy.modflow.mfevt.ModflowEvt(m, surf=ztop, evtr=ET, exdp=extinction_depth, ipakcb=53)'''

### 10. Write inputs and run the model
##### 10.1 Write Inputs

In [ ]:
m.write_input()    #uses the package objects created above to actually write the text files, 
                    #and saves to folder that this script is in
                    #after this step you should see the files appear in your folder

##### 10.2 Run the model

In [ ]:
success, mfoutput = m.run_model(pause=False, report=True)
if not success:
    raise Exception('MODFLOW did not terminate normally.')

### 11. Plot the results
#### 11.1 Read in the model outputs
For information on binary files see: http://modflowpy.github.io/flopydoc/binaryfile.html

For information on flow budget files see: http://modflowpy.github.io/flopydoc/binaryfile.html#flopy.utils.binaryfile.CellBudgetFile

In [ ]:
# Read the heads 
headobj = bf.HeadFile(modelname+'.hds')    #reads the binary head file
times = headobj.get_times()                #returns a list of timesteps
head = headobj.get_data

# hds = bf.HeadFile(modelname+'.hds')                   # reads the binary head file
# times = hds.get_times() # returns a list of timesteps

#extract binary data from head file
head = {} #create dictionary to store head data at end of each stress period
frf = {} #create dictionary to store flows through right cell face at end of each stress period
fff = {} #create dictionary to store flows through front cell face at end of each stress period
headobj = flopy.utils.binaryfile.HeadFile(modelname+'.hds') #get head data as python object
budgobj = flopy.utils.binaryfile.CellBudgetFile(modelname+'.cbc') #get flow data as python object

#get data from python objects
head = []
headarray=np.zeros((len(times),ncol,nrow))
frf=np.zeros((len(times),ncol,nrow))
fff=np.zeros((len(times),ncol,nrow))
rowcounter=-1
for i in times:
    rowcounter+=1
    temphead = headobj.get_data(totim=i) #append heads to head list for each stress per
    headarray[rowcounter,:,:] = temphead
    tempfrf = np.squeeze(budgobj.get_data(text='flow right face',totim=i))
    # print(tempfrf)
    frf[rowcounter,:,:] = tempfrf
    tempfff = np.squeeze(budgobj.get_data(text='FLOW FRONT FACE',totim=i))
    fff[rowcounter,:,:] = tempfff

# extract and plot head time series at specific points
time_series1 = headarray[:,domain_centerpt[1],domain_centerpt[2]]
time_series2 = headarray[:,well_loc[1],well_loc[2]]


plt.subplot(1,2,1)
plt.title("Head at well",fontweight='bold')
plt.xlabel('time (years)',fontweight='bold')
plt.ylabel('head (m)',fontweight='bold')
plt.xticks([1,20,40,60,80,100], ['0', '20', '40','60','80','100'])
plt.plot(elyears, time_series2, 'r-') #plot the time series with points at each record

plt.subplot(1, 2, 2)
plt.title("Head at midpoint",fontweight='bold')
plt.xlabel('time (years)',fontweight='bold')
plt.ylabel('head (m)',fontweight='bold')
plt.xticks([1,20,40,60,80,100], ['0', '20', '40','60','80','100'])
plt.plot(elyears, time_series1, 'r-') #plot the time series with points at each record
plt.tight_layout()

In [ ]:
# find the average at midpoint
testavg = []
x=[]
for i in range(200):
    x = (time_series1[i+1]-time_series1[i])/2
    testavg.append(x)
print(testavg)

In [ ]:
for i in range(nrow):
    hgrad =(headarray[0,25,i]-headarray[0,25,-1])/Lx
    print("Head Gradient", hgrad, "m/m") 

### Plot head distributions after initial steady state, after final off pumping period, and at end of simulation

In [ ]:
## Plot head cross section through well after initial steady state, after final off pumping period, and at end of simulation

In [ ]:

#plot head distribution at years 0, half of sim time, end of sim time
plt.plot(np.arange(nrow),headarray[0,20,:],'r',label='steady state')
plt.plot(np.arange(nrow),headarray[-2,20,:],'b',label='final off')
plt.plot(np.arange(nrow),headarray[-1,20,:],'g',label='final time')
plt.xlabel('Column number')
plt.ylabel('Head (m)')
plt.title("Head Across Transect",fontweight='bold')
plt.legend()

In [ ]:
plt.plot(np.arange(nrow),headarray[0,25,:],'r',label='steady state')
plt.plot(np.arange(nrow),headarray[-2,25,:],'b',label='final off')
plt.plot(np.arange(nrow),headarray[-1,25,:],'g',label='final time')
plt.xlabel('Column number')
plt.ylabel('Head (m)')
plt.title("Head Across Center Transect",fontweight='bold')
plt.legend()

In [ ]:
#plot head distribution at years 0, half of sim time, end of sim time
fig1 = plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0) #use modelmap to attach plot to model
# grid = modelmap.plot_grid() #plot model grid
qm = modelmap.plot_ibound()  
contour_levels = np.linspace(headarray[0].min(),headarray[0].max(),11) #set contour levels for contouring head
head_contours = modelmap.contour_array(headarray[0,:,:], levels=contour_levels) #create head contours   
tempfrf3 = frf[0,:,:].reshape(1,ncol,nrow)
tempfff3 = fff[0,:,:].reshape(1,ncol,nrow)
temphead3 = headarray[0,:,:].reshape(1,ncol,nrow)
flows = modelmap.plot_discharge(tempfrf3, tempfff3, head=temphead3) #create discharge arrows
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.yticks([1, 100, 200, 300, 400,500], ['500', '400', '300','200','100','0'])
plt.title('Head (m) at initial steady state', fontsize = 15, fontweight = 'bold')
plt.colorbar(head_contours,aspect=5)

fig2 = plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0) #use modelmap to attach plot to model
# grid = modelmap.plot_grid() #plot model grid
qm = modelmap.plot_ibound()  
contour_levels = np.linspace(headarray[0].min(),headarray[0].max(),11) #set contour levels for contouring head
head_contours = modelmap.contour_array(headarray[-1,:,:], levels=contour_levels) #create head contours   
tempfrf1 = frf[-1,:,:].reshape(1,ncol,nrow)
tempfff1 = fff[-1,:,:].reshape(1,ncol,nrow)
temphead1 = headarray[-1,:,:].reshape(1,ncol,nrow)
flows = modelmap.plot_discharge(tempfrf1, tempfff1, head=temphead1) #create discharge arrows
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.yticks([1, 100, 200, 300, 400,500], ['500', '400', '300','200','100','0'])
plt.title('Head (m) at end of simulation', fontsize = 15, fontweight = 'bold')
plt.colorbar(head_contours,aspect=5)

fig3 = plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0) #use modelmap to attach plot to model
# grid = modelmap.plot_grid() #plot model grid
qm = modelmap.plot_ibound()  
contour_levels = np.linspace(headarray[0].min(),headarray[0].max(),11) #set contour levels for contouring head
head_contours = modelmap.contour_array(headarray[-2,:,:], levels=contour_levels) #create head contours   
tempfrf2 = frf[-2,:,:].reshape(1,ncol,nrow)
tempfff2 = fff[-2,:,:].reshape(1,ncol,nrow)
temphead2 = headarray[-2,:,:].reshape(1,ncol,nrow)
flows = modelmap.plot_discharge(tempfrf2, tempfff2, head=temphead2) #create discharge arrows
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.yticks([1, 100, 200, 300, 400,500], ['500', '400', '300','200','100','0'])
plt.title('Head (m) at end of final off period', fontsize = 15, fontweight = 'bold')
plt.colorbar(head_contours,aspect=5)

### Plot drawdown

In [ ]:
#plot change in head distribution between steady state and final time
fig1 = plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0) #use modelmap to attach plot to model
# grid = modelmap.plot_grid() #plot model grid
qm = modelmap.plot_ibound()  
contour_min = np.min(headarray[0,:,:]-headarray[-1,:,:])
contour_max = np.max(headarray[0,:,:]-headarray[-1,:,:])
contour_levels = np.linspace(contour_min,contour_max,21) #set contour levels for contouring head
head_contours = modelmap.contour_array(headarray[0,:,:]-headarray[-1,:,:], levels=contour_levels) #create head contours   
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.yticks([1, 100, 200, 300, 400,500], ['500', '400', '300','200','100','0'])
#plt.gca().invert_yaxis()
plt.title('Drawdown (m) from steady state to final time', fontsize = 15, fontweight = 'bold')
plt.colorbar(head_contours,aspect=5)

#plot change in head distribution between steady state and final time
fig1 = plt.figure(figsize=(10,10)) #create 10 x 10 figure
modelmap = flopy.plot.PlotMapView(model=m, layer=0) #use modelmap to attach plot to model
# grid = modelmap.plot_grid() #plot model grid
qm = modelmap.plot_ibound()  
contour_min = np.min(headarray[-2,:,:]-headarray[-1,:,:])
contour_max = np.max(headarray[-2,:,:]-headarray[-1,:,:])
contour_levels = np.linspace(contour_min,contour_max,21) #set contour levels for contouring head
head_contours = modelmap.contour_array(headarray[-2,:,:]-headarray[-1,:,:], levels=contour_levels) #create head contours   
plt.xlabel('Lx (m)',fontsize = 14)
plt.ylabel('Ly (m)',fontsize = 14)
plt.yticks([1, 100, 200, 300, 400,500], ['500', '400', '300','200','100','0'])
#plt.gca().invert_yaxis()
plt.title('Drawdown (m) from final off to final time', fontsize = 15, fontweight = 'bold')
plt.colorbar(head_contours,aspect=5)